# <center>Notebook de démonstration sur AWS SageMaker</center>

### Présentation des données

`bank_clean.csv`

In [1]:
# Import des librairies
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                   
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

# Définir le rôle IAM
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
# Conteneurs disponibles pour XGBOOST
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # chaque région a son propre container
my_region = boto3.session.Session().region_name # selectionner la région de l'instance
print("Succès - l'instance de SageMaker est dans la region " + my_region + ". Utilisez le conteneur " + containers[my_region] + " pour le endpoint SageMaker.")

Succès - l'instance de SageMaker est dans la region us-east-2. Utilisez le conteneur 825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest pour le endpoint SageMaker.


In [4]:
# Création d'un compartiment S3 pour stocker les données de ce tutoriel
bucket_name = 'meetup-prez-brest' # <--- VARIABLE A CHANGER AVEC VOTRE PROPRE NOM UNIQUE
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket, création réussie.')
except Exception as e:
    print('S3 erreur: ',e)

S3 bucket, création réussie.


In [5]:
# Téléchargement des données sur notre instance SageMaker
try:
  urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
  print('Succès: téléchargement de bank_clean.csv effectué.')
except Exception as e:
  print('Erreur chargement des données: ',e)

# Chargement des données pour utilisation par XGBOOST
try:
  model_data = pd.read_csv('./bank_clean.csv',index_col=0)
  print('Succès: données importées dans le dataframe.')
except Exception as e:
    print('Erreur chargement des données: ',e)

Succès: téléchargement de bank_clean.csv effectué.
Succès: données importées dans le dataframe.


In [6]:
# Réorganisation des données
# On divise en jeu d'entraînement et jeu de test
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [7]:
# Reformatage de l'en-tête de la première colonne des données d'entrainement
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
# Chargement des données du compartiment S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [8]:
# Création d'une session sagemaker et initialisation du modèle XGB
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(containers[my_region],role, instance_count=1, instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [9]:
# On fit le modèle avec le jeu d'entraînement
xgb.fit({'train': s3_input_train})

2021-04-29 08:49:48 Starting - Starting the training job...
2021-04-29 08:50:18 Starting - Launching requested ML instancesProfilerReport-1619686188: InProgress
......
2021-04-29 08:51:19 Starting - Preparing the instances for training.........
2021-04-29 08:52:39 Downloading - Downloading input data...
2021-04-29 08:53:19 Training - Training image download completed. Training in progress..Arguments: train
[2021-04-29:08:53:14:INFO] Running standalone xgboost training.
[2021-04-29:08:53:14:INFO] Path /opt/ml/input/data/validation does not exist!
[2021-04-29:08:53:14:INFO] File size need to be processed in the node: 3.38mb. Available memory size in the node: 8423.95mb
[2021-04-29:08:53:14:INFO] Determined delimiter of CSV input is ','
[08:53:14] S3DistributionType set as FullyReplicated
[08:53:14] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[08:53:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nod

In [10]:
# Déploiement du modèle
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

---------------!

In [11]:
# Récupération des données de tests
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #Charger les data dans un array
xgb_predictor.serializer = CSVSerializer() # selectionner le type de serialiser
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # transformer les predictions en array
print(predictions_array.shape)

(12357,)


In [12]:
# Visualisation d ela prédiction avec une matrice de confusion
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Taux de classification: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Prediction", "Non Acheteur", "Acheteur"))
print("Observé")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("Non Acheteur", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Acheteur", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Taux de classification: 89.5%

Prediction     Non Acheteur   Acheteur
Observé
Non Acheteur   90% (10769)    37% (167)
Acheteur        10% (1133)     63% (288) 



In [13]:
# Suppression des ressources
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint_name)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'HR7ADRSSA82VEHQ7',
   'HostId': '4crVmmP9H8pB+7G285RQqOz0+CrajbZnKmCAUzWDDzbbHINA8HxhXAoIxuI+L3uBI84eCiowblQ=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '4crVmmP9H8pB+7G285RQqOz0+CrajbZnKmCAUzWDDzbbHINA8HxhXAoIxuI+L3uBI84eCiowblQ=',
    'x-amz-request-id': 'HR7ADRSSA82VEHQ7',
    'date': 'Thu, 29 Apr 2021 09:03:16 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2021-04-29-08-49-48-784/rule-output/ProfilerReport-1619686188/profiler-output/profiler-reports/LoadBalancing.json'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2021-04-29-08-49-48-784/rule-output/ProfilerReport-1619686188/profiler-output/profiler-reports/StepOutlier.json'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2021-04-29-08-49-48-784/rule-output/ProfilerReport-1619686188/profil